# Árbol de decisión - Clasificación

In [1]:
#Primero hay que llamar a unas librerías que el programa necesitará.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

import random

# Preparacion de datos
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Se lee el archivo
archivo_data = pd.read_csv('hour.csv', delimiter=' *, *', engine='python')
#Verificamos los 5 primeros registros
archivo_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [3]:
# Verificamos si existen datos nulos
archivo_data.isna().any()

instant       False
dteday        False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
windspeed     False
casual        False
registered    False
cnt           False
dtype: bool

In [4]:
#Para comprobar que se ha leído bien, podemos lista las variables en el fichero y sacar los principales estadísticos
archivo_data.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [5]:
#Principales estadísticos
archivo_data.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [6]:
# Creamos una copia del dataframe original
archivo_data_copia = archivo_data
# Eliminamos las 2 columnas, ya que no se consideraran en el análisis
archivo_data_copia = archivo_data_copia.drop(['instant','dteday'], axis=1)
#verificamos los 5 primeros registros
archivo_data_copia.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [7]:
#Principales estadísticos
archivo_data_copia.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [8]:
#Calculamos la mediana de cada características
archivo_data_copia.median()

season          3.0000
yr              1.0000
mnth            7.0000
hr             12.0000
holiday         0.0000
weekday         3.0000
workingday      1.0000
weathersit      1.0000
temp            0.5000
atemp           0.4848
hum             0.6300
windspeed       0.1940
casual         17.0000
registered    115.0000
cnt           142.0000
dtype: float64

In [9]:
#Calculamos los cuartiles:
# Cuartil 1  es de 25%
# Cuartil 2  es de 50%
# Cuartil 3  es de 75%
q25=np.percentile(archivo_data_copia[['registered']], 25)
q75=np.percentile(archivo_data_copia[['registered']], 75)
RIC=q75-q25
LI=q25-1.5*RIC
LS=q75+1.5*RIC
#Verificamos los cortes:
# LI: Limite Inferior
# SL: Limite superior
# Los valores que estan fuera de este rango, se lo considera valores atípicos
print(LI,LS)

-245.0 499.0


In [10]:
MIN=archivo_data_copia[['registered']].min()
MAX=archivo_data_copia[['registered']].max()
print(MIN,MAX)

# df_LI: Calcula 0 datos como datos atípicos por el límite inferior
df_LI=archivo_data_copia[archivo_data_copia['registered']>=LI]
# df_LS: Calcula 886 que son datos atipicos por ser mayor al Límite Superoir
# Al utilizar el dataframe, restara el valor total con los atípicos
df_LS=archivo_data_copia[archivo_data_copia['registered']<=LS]

registered    0
dtype: int64 registered    886
dtype: int64


In [11]:
#Creo una copia del dataframe archivo_data_copia
archivo_data_copia1 = archivo_data_copia
# Se crea una nueva columna y los valores que son "atipicos" son reemplazos con el cuantil 3
archivo_data_copia1['registered2'] = archivo_data_copia1['registered'].where(archivo_data_copia1['registered']<=LS,q75)

In [12]:
archivo_data_copia1.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,registered2
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088,137.728580
std,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599,116.606252
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000,34.000000
50%,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000,115.000000
75%,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000,220.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000,499.000000


In [13]:
archivo_data_copia1.median()

season           3.0000
yr               1.0000
mnth             7.0000
hr              12.0000
holiday          0.0000
weekday          3.0000
workingday       1.0000
weathersit       1.0000
temp             0.5000
atemp            0.4848
hum              0.6300
windspeed        0.1940
casual          17.0000
registered     115.0000
cnt            142.0000
registered2    115.0000
dtype: float64

In [14]:
q25=np.percentile(archivo_data_copia1[['registered2']], 25)
q75=np.percentile(archivo_data_copia1[['registered2']], 75)
RIC=q75-q25
LI=q25-1.5*RIC
LS=q75+1.5*RIC
print(LI,LS)

-245.0 499.0


In [15]:
MIN=archivo_data_copia1[['registered2']].min()
MAX=archivo_data_copia1[['registered2']].max()
print(MIN,MAX)

df_LI1=archivo_data_copia1[archivo_data_copia1['registered2']>=LI]
#Verificamos que los valores han mejorado. De 886 ya existen 499
df_LS1=archivo_data_copia1[archivo_data_copia1['registered2']<=LS]

registered2    0
dtype: int64 registered2    499
dtype: int64


In [16]:
# Realizamos el mismo analisis para la caracteristica "casual"
q25=np.percentile(archivo_data_copia1[['casual']], 25)
q75=np.percentile(archivo_data_copia1[['casual']], 75)
RIC=q75-q25
LI=q25-1.5*RIC
LS=q75+1.5*RIC
print(LI,LS)

-62.0 114.0


In [17]:
MIN=archivo_data_copia1[['casual']].min()
MAX=archivo_data_copia1[['casual']].max()
print(MIN,MAX)

df_LI1=archivo_data_copia1[archivo_data_copia1['casual']>=LI]
# df_LS1: ha encontrado 367 datos atípicos
df_LS1=archivo_data_copia1[archivo_data_copia1['casual']<=LS]

casual    0
dtype: int64 casual    367
dtype: int64


In [18]:
# Se crea una nueva columna para casual. Para los datos atípicos son reemplazado con el cuantil 3
archivo_data_copia1['casual2'] = archivo_data_copia1['casual'].where(archivo_data_copia1['casual']<=LS,q75)

In [19]:
archivo_data_copia1.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,registered2,casual2
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088,137.728580,26.631970
std,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599,116.606252,27.206421
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000,34.000000,4.000000
50%,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000,115.000000,17.000000
75%,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000,220.000000,48.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000,499.000000,114.000000


In [20]:
archivo_data_copia1.median()

season           3.0000
yr               1.0000
mnth             7.0000
hr              12.0000
holiday          0.0000
weekday          3.0000
workingday       1.0000
weathersit       1.0000
temp             0.5000
atemp            0.4848
hum              0.6300
windspeed        0.1940
casual          17.0000
registered     115.0000
cnt            142.0000
registered2    115.0000
casual2         17.0000
dtype: float64

In [21]:
q25=np.percentile(archivo_data_copia1[['casual2']], 25)
q75=np.percentile(archivo_data_copia1[['casual2']], 75)
RIC=q75-q25
LI=q25-1.5*RIC
LS=q75+1.5*RIC
print(LI,LS)

-62.0 114.0


In [22]:
MIN=archivo_data_copia1[['casual2']].min()
MAX=archivo_data_copia1[['casual2']].max()
print(MIN,MAX)

df_LI1=archivo_data_copia1[archivo_data_copia1['casual2']>=LI]
#Verificamos que los valores han mejorado de 367 bajo a 114
df_LS1=archivo_data_copia1[archivo_data_copia1['casual2']<=LS]

casual2    0
dtype: int64 casual2    114
dtype: int64


In [23]:
df_LS1.count()

season         17379
yr             17379
mnth           17379
hr             17379
holiday        17379
weekday        17379
workingday     17379
weathersit     17379
temp           17379
atemp          17379
hum            17379
windspeed      17379
casual         17379
registered     17379
cnt            17379
registered2    17379
casual2        17379
dtype: int64

In [24]:
# Dividir conjuntos de datos
from sklearn.model_selection  import train_test_split

# Realizar los grupos de CNT
archivo_data_copia1['c_cnt'] = np.where(archivo_data_copia1['cnt']<=20,1,2)

#Muestreo
#train, test = train_test_split(archivo_data_copia1, test_size = 0.3)
X = archivo_data_copia1[['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed','casual2','registered2']].values
y = archivo_data_copia1['c_cnt']


feature_list = ['season','yr','mnth','hr','holiday','weekday','workingday','weathersit','temp','atemp','hum','windspeed','casual2','registered2']

In [25]:
archivo_data_copia1.groupby(['c_cnt']).count()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,registered2,casual2
c_cnt,,,,,,,,,,,,,,,,,
1,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994,2994
2,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385,14385


In [26]:
# Se crea una semilla
random.seed(100)

In [27]:
## Finalmente, dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)
#train, test = train_test_split(df_LS1, test_size=0.30)

# Creación del árbol

In [28]:
#---MODELO
arbol=DecisionTreeClassifier()

In [29]:
#Ajuste
arbol.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [30]:
pred_test=arbol.predict(X_test)
pred_test

array([2, 2, 2, ..., 2, 1, 2])

In [31]:
pred_train=arbol.predict(X_train)
pred_train

array([2, 2, 1, ..., 2, 2, 2])

In [32]:
arbol.predict_proba(X_train)

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [38]:
round(arbol.score(X_train,y_train),2)

1.0

In [34]:
#---Matriz de CONFUSIÓN -- Precision Global
#--Testing
sum(pred_test==y_test)/(y_test.count())

0.9980820866896817

In [46]:
#--Training
sum(pred_train==y_train)/(y_train.count())

1.0